<a href="https://colab.research.google.com/github/Xiaoyan-Adele/Consumer-Subsidies-and-French-Industrial-Policy-for-EV/blob/master/Topic%20Modeling/LDA_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
Topic model aims at discover the uncovering structure in a collection of texts. In project, we would experiment with documents released on using **keyword research "véhicules électriques"**. The corpus was scrapped from the the French government website. 

Our topic modeling used the **Lantent Dirichlet Allocation (LDA)** method using the Gensim library in jupyter notebook editor.

The Colab version is for more convenient editing of the code description and results presentation.



## LDA Implementation Outlines


1. Library import
2. Data cleaning
3. Preparing data for LDA analysis
4. Model training and result visualization
5. Model tuning and final results presentation




In [39]:
!pip install pyLDAvis
!pip install spacy==2.0.18
import spacy
!python -m spacy download fr_core_news_sm


    Linking successful
    /usr/local/lib/python3.6/dist-packages/fr_core_news_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/fr_core_news_sm

    You can now load the model via spacy.load('fr_core_news_sm')



### 1. Import most of the necessary libraries to set up the operation environment

In [0]:
import re
import string
import numpy as np
import pandas as pd
import os
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  
# don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [41]:
# Upload from local
from google.colab import files
uploaded = files.upload()
print("len(uploaded.keys():", len(uploaded.keys()))

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))



Saving handpicked-article.csv to handpicked-article.csv
len(uploaded.keys(): 1
User uploaded file "handpicked-article.csv" with length 129235 bytes


In [42]:
# Get dataframe of the tweets in the labeled dataset 
evarticles = pd.read_csv('handpicked-article.csv',encoding = 'utf-8',sep=',')

#print head
evarticles.head(2)

,content
0,\n QU’EST-CE QUI A CHANGÉ ?\n\nLa prime à la ...
1,\n\n\n \n \n \n \n ...


### 2. Clean the text data

Here we conducted a series of corpus preprocessing accustomed to the characteristics of our dataset. Besides standard practices such as transforming to lowerase, removing numbers, spliting and deleting punctuations, this project took into consideration that we were to model with a corpus that has excessive phrases such as "\n" and the text was in French. Thus we adjust the stopwords accordingly and switched from the initially chosen sklearn library to gensim, since we did not figure out how to build the word dictionary in French with sklearn. 

In the last line fo this section, we inspected the results of our preprocessing and sort them into a seperate column. By doing this, we can compare the preprocessed text with the original one and get an intuitive assessment of the work.

In [43]:
# Load the regular expression library
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 

from spacy.lang.fr.stop_words import STOP_WORDS


# Remove punctuation
evarticles['content_processed'] = evarticles['content'].map(lambda x: re.sub('[,-:\.!?%‎€]', '', x))

#remove and \n
evarticles['content_processed'] = evarticles['content_processed'].map(lambda x: re.sub('\n','', x))
evarticles['content_processed'] = evarticles['content_processed'].map(lambda x: re.sub('%','', x))
evarticles['content_processed'] = evarticles['content_processed'].map(lambda x: re.sub('€','', x))

#remove numbers, excessive white space
space_pattern = '\s+'
evarticles['content_processed'] = evarticles['content_processed'].map(lambda x: re.sub(space_pattern,' ', x))
evarticles['content_processed'] = evarticles['content_processed'].map(lambda x: re.sub('[0-9]','', x))

# Convert the titles to lowercase
evarticles['content_processed'] = evarticles['content_processed'].map(lambda x: x.lower().split())


# Initialize spacy 'fr' model, keeping only tagger component (for efficiency)
nlp = spacy.load('fr_core_news_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


evarticles['content_processed'] = lemmatization(evarticles['content_processed'], allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

#remove stop words
def remove_stopwords(X_list):
    filtered_list =[]
    for it in X_list:
        temp = []
        for word in it:
            if(word not in STOP_WORDS):
                temp.append(word)
        filtered_list.append(temp)
        temp=[] 
    return filtered_list

evarticles['content_processed'] = remove_stopwords(evarticles['content_processed'])

#take out numbers and 
evarticles.head(10)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,content,content_processed
0,\n QU’EST-CE QUI A CHANGÉ ?\n\nLa prime à la ...,"[estce, changer, prime, conversion, aide, fina..."
1,\n\n\n \n \n \n \n ...,"[juillet, environnement, état, engager, pollut..."
2,\n\n\n \n \n \n \n ...,"[prime, conversion, renforcer, accompagner, au..."
3,\n\n\n \n \n \n \n ...,"[novembre, transport, projet, loi, penser, mob..."
4,\n\n5 décembre 2018 - CommuniquéGroupe VALE en...,"[décembre, communiquégroupe, val, nouvellecalé..."
5,\n\n\n \n \n \n \n ...,"[octobre, ambition, échelle, européen, véhicul..."
6,\n\n4 juillet 2019 - DiscoursDiscours à l'occa...,"[juillet, discoursdiscour, occasion, visiter, ..."
7,\n\n\n \n [Grand Défi] Développement du st...,"[grand, défi, développement, stockage, énergie..."
8,\n\n\n \n \n \n \n ...,"[voitur, autonome, rapport, développer, filièr..."
9,\n\n\n \n \n \n \n ...,"[décembre, change, janvier, interdiction, plas..."


In [44]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(evarticles['content_processed'], min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[evarticles['content_processed']], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[evarticles['content_processed'][0]]])

['estce', 'changer', 'prime_conversion', 'aide', 'financier', 'etat', 'permettre', 'acheter', 'louer', 'véhiculer', 'polluer', 'occasion', 'condition', 'mettre', 'casse', 'ancien', 'véhiculequel', 'montant', 'prime', 'montant', 'prime', 'vari', 'fonction', 'véhicule', 'acheter', 'louer', 'revenu', 'dépendre', 'catégorie', 'véhicule', 'revenu', 'fiscal', 'référence', 'trouver', 'avis', 'impôt', 'revenu', 'nombre', 'kilomètre', 'effectuer', 'travailler', 'monter', 'prime', 'élevé', 'famille', 'modeste', 'faire', 'kilomètre', 'jour', 'travail', 'monter', 'prime', 'type', 'véhicule', 'acheter', 'montant', 'prime', 'véhicul', 'électrique', 'modeler', 'hybride', 'rechargeable', 'occasion', 'véhicule', 'air', 'occasion', 'jusqu’', 'simuler', 'montant', 'prime', 'conversionpourquoi', 'mesure', 'remplacer', 'vieux', 'voiture', 'polluant', 'voiture', 'récent', 'bon', 'planète', 'véhicule', 'récent', 'polluant', 'permettre', 'air', 'meilleur', 'qualité', 'ville', 'réduire', 'émission_gaz_serre', 

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [0]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]


In [0]:
# Form Bigrams
evarticles_bigrams = make_bigrams(evarticles['content_processed'])


In [47]:
evarticles_preprocessed = evarticles['content_processed']


# Create Dictionary
id2word = corpora.Dictionary(evarticles_preprocessed)

# Create Corpus
texts = evarticles_preprocessed

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 2), (1, 8), (2, 1), (3, 1), (4, 3), (5, 1), (6, 8), (7, 3), (8, 6), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 2), (15, 1), (16, 2), (17, 1), (18, 5), (19, 1), (20, 2), (21, 1), (22, 1), (23, 5), (24, 4), (25, 1), (26, 1), (27, 4), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 4), (39, 1), (40, 1), (41, 1), (42, 5), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 3), (51, 1), (52, 1), (53, 4), (54, 4), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 2), (64, 2), (65, 1), (66, 1), (67, 1), (68, 3), (69, 4), (70, 3), (71, 1), (72, 2), (73, 1), (74, 1), (75, 1), (76, 1), (77, 2), (78, 1), (79, 3), (80, 1), (81, 1), (82, 1), (83, 4), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 2), (90, 1), (91, 1), (92, 1), (93, 4), (94, 1), (95, 1), (96, 2), (97, 1), (98, 5), (99, 1), (100, 1), (101, 1), (102, 6), (103, 1), (104, 1), (105, 3), (106, 1), (107, 5), (108, 2), (109, 2), (110, 1)

In [21]:
id2word[5]

'aide'

### 4. Build the model

We consulted the code written by Selva Prabhakaran on *Topic Modeling with Gensim (Python)* and set the hyperparameters accordingly. After completing the first model, I fed it with the processed text and compute the coherence score and visualized its results.


**Results visualization**

the interactive chart built with pyLDAvis package gave very good representation of topics learned from the text and their associated keywords.

**Topic coherence score**

the score provided a easy-to-read judgement on the quality of our LDA model. 

In [0]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [73]:
# Print the Keyword in the 20 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.018*"corse" + 0.016*"énergie" + 0.015*"énergétique" + 0.012*"objectif" + '
  '0.012*"transition" + 0.011*"gouvernement" + 0.008*"permettre" + '
  '0.007*"électricité" + 0.007*"climatique" + 0.007*"projet"'),
 (1,
  '0.017*"énergie" + 0.011*"nucléaire" + 0.009*"énergétique" + 0.009*"prix" + '
  '0.008*"renouvelable" + 0.008*"objectif" + 0.008*"hausse" + '
  '0.007*"gouvernement" + 0.007*"ici" + 0.007*"euro"'),
 (2,
  '0.024*"industrie" + 0.010*"industriel" + 0.007*"français" + 0.007*"faire" + '
  '0.007*"filière" + 0.006*"permettre" + 0.006*"investissement" + '
  '0.006*"france" + 0.006*"véhicule" + 0.005*"entreprise"'),
 (3,
  '0.024*"mobilité" + 0.020*"transport" + 0.018*"loi" + 0.011*"solution" + '
  '0.011*"véhicule" + 0.009*"grand" + 0.009*"orientation" + 0.009*"prime" + '
  '0.009*"ministre" + 0.008*"électrique"'),
 (4,
  '0.017*"véhicule" + 0.015*"européen" + 0.013*"émission" + 0.011*"réduction" '
  '+ 0.009*"objectif" + 0.009*"ambition" + 0.009*"octobre" + 0.007*"perme

In [74]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.079539 -0.127474       1        1  30.898741
0     -0.151352 -0.056438       2        1  18.166676
5      0.120233  0.001919       3        1  15.438096
1     -0.120298 -0.073958       4        1  11.950430
3     -0.060388  0.117458       5        1   9.897977
9      0.002743  0.025057       6        1   5.660804
7     -0.012088  0.098585       7        1   5.532823
4      0.041275  0.007394       8        1   2.441190
6      0.048805  0.001495       9        1   0.006749
8      0.051530  0.005962      10        1   0.006515, topic_info=            Term       Freq      Total Category  logprob  loglift
619    industrie  66.000000  66.000000  Default  30.0000  30.0000
178     véhicule  65.000000  65.000000  Default  29.0000  29.0000
133        prime  32.000000  32.000000  Default  28.0000  28.0000
728        corse  28.000000  28.000000  Default  27.0000  27.0000
193      énergie  53.000000  53.000000  Default  26.0000  26.0000
..           ...        ...        ...      ...      ...      ...
275  orientation   0.000242  16.708702  Topic10  -7.7230  -1.5018
316    transport   0.000245  32.253384  Topic10  -7.7121  -2.1486
728        corse   0.000245  28.086727  Topic10  -7.7146  -2.0127
438        grand   0.000244  28.332394  Topic10  -7.7152  -2.0220
312   territoire   0.000244  26.429998  Topic10  -7.7175  -1.9548

[694 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
2143      4  0.913164   abaisser
663       2  0.365497    aborder
663       7  0.365497    aborder
664       2  0.828076    aboutir
664       4  0.207019    aboutir
...     ...       ...        ...
945       2  0.865308   éventuel
1051      2  0.412039  évolution
1051      3  0.206020  évolution
1051      4  0.412039  évolution
2525      7  0.809160  évènement

[1019 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 6, 2, 4, 10, 8, 5, 7, 9])

In [75]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=evarticles_preprocessed, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

print('\nCoherence Score: ', coherence_lda)




Coherence Score:  0.39153460603519774


### 5. Parameters tuning and final results presentation

After receiving the initial coherence score 0.4523, I consulted code by Shashank Kapadia to define a series of sensitivity tests which helpto determine optimal number of topics (K), LDA document_topic density (alpha) and the word-topic density (eta). Once the function is created, I looped it over the range of topics, alpha and beta with the eqdm package.

After validation, the final LDA model achieves a coherence score of 0.4784

In [0]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b,
                                           per_word_topics=True)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=evarticles_preprocessed, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [77]:
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 20
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()


  0%|          | 0/540 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)

100%|██████████| 540/540 [14:24<00:00,  1.55s/it]
541it [14:26,  1.67s/it]                         
542it [14:27,  1.67s/it]
543it [14:29,  1.67s/it]
544it [14:31,  1.68s/it]
545it [14:33,  1.68s/it]
546it [14:34,  1.74s/it]
547it [14:36,  1.71s/it]
548it [14:38,  1.77s/it]
549it [14:40,  1.76s/it]
550it [14:41,  1.75s/it]
551it [14:44,  1.87s/it]
552it [14:45,  1.81s/it]
553it [14:47,  1.81s/it]
554it [14:49,  1.79s/it]
555it [14:51,  1.79s/it]
556it [14:52,  1.81s/it]
557it [14:54,  1.80s/it]
558it [14:56,  1.79s/it]
559it [14:58,  1.78s/it]
560it [15:00,  1.78s/it]
561it [15:01,  1.82s/it]
562it [15:03,  1.78s/it]
563it [15:05,  1.75s/it]
564it [15:07,  1.76s/it]
565it [15:08,  1.75s/it]
566it [15:10,  1.76s/it]
567it [15:12,  1.73s/it]
568it [15:13,  1.71s/it]
569it [15:15,  1.70s/it]
570i

In [0]:
print(type(model_results))

<class 'dict'>


In [0]:
#optimized model
lda_model_new = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.61,
                                           eta=0.91)

In [79]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model_new, corpus, id2word)

LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.060477  0.021599       1        1  30.294231
0      0.028968  0.041216       2        1  18.839373
9      0.001820 -0.009041       3        1  12.933034
1      0.015096  0.014038       4        1  10.419427
5     -0.003013 -0.020548       5        1   9.888078
3      0.009401 -0.008400       6        1   9.527157
7      0.007506 -0.008007       7        1   5.637510
4     -0.000310 -0.011780       8        1   2.086597
8      0.000509 -0.009571       9        1   0.187295
6      0.000501 -0.009506      10        1   0.187295, topic_info=            Term       Freq      Total Category  logprob  loglift
619    industrie  39.000000  39.000000  Default  30.0000  30.0000
728        corse  15.000000  15.000000  Default  29.0000  29.0000
178     véhicule  32.000000  32.000000  Default  28.0000  28.0000
193      énergie  25.000000  25.000000  Default  27.0000  27.0000
268     mobilité  17.000000  17.000000  Default  26.0000  26.0000
..           ...        ...        ...      ...      ...      ...
69         faire   0.006357  18.705545  Topic10  -7.8152  -1.7068
163   transition   0.006353  16.526545  Topic10  -7.8158  -1.5836
330  énergétique   0.006350  18.967751  Topic10  -7.8162  -1.7218
264          loi   0.006344  11.874786  Topic10  -7.8172  -1.2544
460     national   0.006344  13.314076  Topic10  -7.8172  -1.3689

[637 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
2143      4  0.376978     abaisser
663       2  0.350230      aborder
664       2  0.497100      aboutir
664       4  0.248550      aboutir
194       1  0.499743  accompagner
...     ...       ...          ...
331       6  0.180293         état
2109      1  0.337076      évaluer
1051      1  0.168656    évolution
1051      2  0.337312    évolution
1051      4  0.337312    évolution

[890 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 10, 2, 6, 4, 8, 5, 9, 7])

In [80]:
coherence_model_lda_new = CoherenceModel(model=lda_model_new, texts=evarticles_preprocessed, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda_new.get_coherence()

print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.41428382760779153
